# Analysing statistics for PGA Tour players in the 2022 season

In this notebook, I will be using data from __[Advanced Sports Analytics](https://www.advancedsportsanalytics.com/pga-raw-data)__ to look at different statistics for PGA Tour players in the 2022 season. 

0. The first step is to import the necessary libraries for our analysis

In [1]:
import pandas as pd
import numpy as np
import datetime

1. The next step is to import the data and inspect it to look at the different columns, data types, and to check for null values. 

In [2]:
# Import the data into a pandas dataframe
pga_data = pd.read_csv('pgatour_22.csv')
#Print the first 5 rows the data to see how the data is arranged into rows and columns
print('Initial look at raw dataset:\n',pga_data.head())
#Print the data types of our columns
print('-'*100)
print('Data types for raw dataset:\n',pga_data.dtypes)
#Print the number of null values in each column
print('-'*100)
print('Null values in each column:\n',pga_data.isna().sum())

Initial look at raw dataset:
   Player_initial_last  tournament id  player id  hole_par  strokes  hole_DKP  \
0            A. Ancer      401353224       9261       288      289      60.0   
1           A. Hadwin      401353224       5548       288      286      72.5   
2           A. Lahiri      401353224       4989       144      147      21.5   
3             A. Long      401353224       6015       144      151      20.5   
4            A. Noren      401353224       3832       144      148      23.5   

   hole_FDP  hole_SDP  streak_DKP  streak_FDP  ...  purse  season  no_cut  \
0      51.1        56           3         7.6  ...   12.0    2022       0   
1      61.5        61           8        13.0  ...   12.0    2022       0   
2      17.4        27           0         0.0  ...   12.0    2022       0   
3      13.6        17           0         0.4  ...   12.0    2022       0   
4      18.1        23           0         1.2  ...   12.0    2022       0   

   Finish  sg_putt  sg_arg

In [3]:
#Check the player column to see the data it gives us, as we also have a player_intial_last column
print(pga_data.player.head())
print(pga_data.Player_initial_last.head())
print('-'*100)
#Get the number of tournaments we have information about
print('Number of unique tournaments played:',pga_data['tournament name'].nunique())
print('Number of unique players:',pga_data.player.nunique())

0      Abraham Ancer
1        Adam Hadwin
2     Anirban Lahiri
3          Adam Long
4    Alexander Noren
Name: player, dtype: object
0     A. Ancer
1    A. Hadwin
2    A. Lahiri
3      A. Long
4     A. Noren
Name: Player_initial_last, dtype: object
----------------------------------------------------------------------------------------------------
Number of unique tournaments played: 32
Number of unique players: 337


## Data Info

1. It looks like we have 37 columns with information about 67 tournaments and 499 players from the 2018-2022 season.
 
    1. `hole_par` tells us the total shots a player must shoot to score level par accorindg to the amount of rounds they've played (2 rounds ~ 144, 4 rounds ~ 288)
    2. `strokes` tells us the number of strokes the player made on the hole. For example, if a player makes 6 strokes on a par 4, their score for the hole is +2, or 2 strokes over par!
    3. `num_rounds` tells us the number of rounds the player played in a certain tournament. It usually has values of either 2 or 4 (explanation below).
    4. `made_cut` tells us whether a player made the *cut* (1) or not (0). PGA Tour events last 4 days and the playing field is **cut** in half after the end of the 2nd day, meaning players that *made the cut* play 4 rounds, but players that do not make the cut play 2 rounds. It could be that a player was disqualified or had to withdraw during the 1st or 3rd round, but values of 1 or 3 in `num_rounds` would be rare (good to be aware of them though!).
    5. `pos` tells us the position the player finished at the end of the tournament in `float` form, so if a player did not make the cut, it gives a `NaN` value.
    6. `date` gives us the date (normal `yyyy-mm-dd` form) on which a certain row gives information about.
    7. `purse` is the prize money for the tournament in Millions.
    8. `no_cut` tell us whether a tournament had a cut or not, this information is somewhat important as most tournaments **do** have cuts, so we might want to filter these out.
    9. `Finish` gives the same information as `pos` and a little more (tells us whether a player is Tied for a position, didn't make the cut (CUT), withdrew (WD), or got disqualified (DQ)) but displays the information in a way familiar to golfers (CUT meaning the player didn't make the cut, Tnum meaning the player tied for num-th place.)
    10. `sg-` means **strokes gained** and is a comparison of the player's performance with the rest of the field (Please see https://www.pgatour.com/news/2016/05/31/strokes-gained-defined.html)
        1. `sg_putt` means strokes gained putting.
        2. `sg_arg` means strokes gained around the green (within 30 yards of green) (the *green* is the surface where the hole is located).
        3. `sg_agg` means strokes gained approach the green (any shot hit towards the green that is not a tee shot on a par 4 or par 5, but includes tee shots on par 3s) (a tee shot is the first shot on any hole).
        4. `sg_ott` means strokes gained off the tee.
        5. `sg_t2g` means strokes gained tee to green (`sg_t2g = sg_arg + sg_agg + sg_ott`)
        6. `sg_total = sg_t2g + sg_putt`

## Data Cleaning strategy

1. The naming of the columns is very inconsistent as some column names have title style while some do not, some contain underscores while some do not etc. The naming will have to be fixed. Some of the names of the columns also do not do a great job of explaining what the columns values are representing, especially for non-golfing audiences.

2. The columns that contain the three letter acronyms *FDP, DKP, SDP* have to do with Draft King / Fantasy Drafts, i.e. leagues where followers of the tour can select players to be in their 'fantasy team' and see who gets the most points. This data is of little use to my analysis here as we are MOSTLY INTERESTED IN THE STATS OF THE PLAYERS THEMSELVES.
**These columns (12) will be removed**

3. We have columns in the form `Unnamed: number` which only contain null values.
**These columns (3) will also be removed**

4. There are 2 columns containing information about the Players name: `Player_initial_last` and `Player`.
**This is giving us redundant information and can definitely be reduced into one column**

5. `pos` and `Finish` give us the same information as different data types, but it easier working with numeric data types. Whether a player ties for 2nd place or comes solo 2nd is not of much importance to us. If `made_cut` is False, then the `pos` is `NaN`. Where Finish is `null`, the position column is sometimes `not null` and vice versa. However, where both `Finish` and `pos` are `Null`, the strokes gained columns are also `null`. 
**`Finish` and `pos` are thus both important. The rows where Finish and pos are null also have all the strokes gained columns as null, so these values will have to be filled in using other data. For clarity, if the player did not make a cut, the value in the pos column will be set to 0, to separate the missed cuts data points to data points where we have missing data**

6. `course` contains the course name, city, and state the golf tournament was held in. 
**For easy access to information, we can split the data into 3 columns: `course`, `city`, `state`**

7. There are some tournaments in our data that we are not interested in:
Corales Puntacana Resort & Club Championship, Puerto Rico Open, AT&T Pebble Beach Pro-Am, Mayakoba Golf Classic, and Bermuda Championship will be removed from our data.

In [4]:
# Remove columns with the fantasy leagues information
pga_data = pga_data[pga_data.columns.drop(list(pga_data.filter(regex=\
                                                               '(hole|streak|finish|total)\_[DFS][KD]P')))]
pga_data = pga_data[pga_data.columns.drop(list(pga_data.filter(regex='Unnamed: \d')))]

#Make the made_cut column a boolean dtype column
pga_data['made_cut'] = pga_data.made_cut.replace([0, 1], [False, True])
pga_data['no_cut'] = pga_data.no_cut.replace([0, 1], [False, True])

#split information in course into 3 columns: course, city, and state
course_data = pga_data.course.str.split('-')
pga_data.course = course_data.str.get(0).str.strip().astype('string')
city_state = course_data.str.get(1)
city_state = city_state.str.split(',')
pga_data['city'] = city_state.str.get(0).str.strip().astype('string')
pga_data['state'] = city_state.str.get(1).str.strip().astype('string')

#Use pos and Finish columns to help fill each other
pos_str = pga_data['pos'].astype('object')
pga_data['Finish'].fillna(pos_str, inplace=True)

'''creates a new position column that is the same as pos 
except where Finish is NaN, where it is filled with 0s'''
pga_data['position'] = pga_data.Finish.apply(lambda x: 0 if (str(x) == 'CUT') or (str(x) == 'DQ') or (str(x) == 'WD') or (str(x) == 'MDF') else\
                                         (np.nan if x == np.nan else (str(x).strip('T') if str(x)[0] == 'T' else x))).astype('float')

#Change the dtypes of columns to correct dtypes suiting data
pga_data['tournament name'] = pga_data['tournament name'].astype('string')
pga_data['Finish'] = pga_data['Finish'].astype('string')
pga_data['course'] = pga_data.course.astype('string')
pga_data['date'] = pd.to_datetime(pga_data.date)
pga_data['player'] = pga_data['player'].astype('string')
pga_data['tournament id'] = pga_data['tournament id'].astype('string')
pga_data['player id'] = pga_data['player id'].astype('string')

In [5]:
#Remove tournaments we are not interested in
pga_data = pga_data[(pga_data['tournament name'] != 'Corales Puntacana Resort & Club Championship') \
                    & (pga_data['tournament name'] != 'Puerto Rico Open') & \
                    (pga_data['tournament name'] != 'AT&T Pebble Beach Pro-Am') &\
                   (pga_data['tournament name'] != 'Mayakoba Golf Classic') &\
                   (pga_data['tournament name'] != 'Bermuda Championship')]

#rename column names to match python variable naming convention
pga_data = pga_data.rename(columns = {'tournament id': 'tournament_id', 'player id': 'player_id',\
                                     'n_rounds': 'rounds_played', 'tournament name': 'tournament',\
                                     'Finish': 'finish'})

#get rid of redundant columns
pga_data = pga_data.drop(columns = ['Player_initial_last','hole_par','season','pos'])

In [6]:
#change order of columns in df to make logical sense to reader as they read
cols = pga_data.columns.tolist()
colsdict = dict()
for i in range(len(cols)):
     colsdict[i] = cols[i]
print(colsdict)
cols_order = [1,5,0,6,8,7,18,19,9,10,11,20,4,3,2,12,13,14,15,16,17]
cols = [cols[i] for i in cols_order]
pga_data = pga_data[cols]

{0: 'tournament_id', 1: 'player_id', 2: 'strokes', 3: 'rounds_played', 4: 'made_cut', 5: 'player', 6: 'tournament', 7: 'course', 8: 'date', 9: 'purse', 10: 'no_cut', 11: 'finish', 12: 'sg_putt', 13: 'sg_arg', 14: 'sg_app', 15: 'sg_ott', 16: 'sg_t2g', 17: 'sg_total', 18: 'city', 19: 'state', 20: 'position'}


In [7]:
pd.set_option('display.max_rows', 150)
#Get an idea of the dataset information
print(pga_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3122 entries, 0 to 3675
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   player_id      3122 non-null   string        
 1   player         3122 non-null   string        
 2   tournament_id  3122 non-null   string        
 3   tournament     3122 non-null   string        
 4   date           3122 non-null   datetime64[ns]
 5   course         3122 non-null   string        
 6   city           3122 non-null   string        
 7   state          3122 non-null   string        
 8   purse          3122 non-null   float64       
 9   no_cut         3122 non-null   bool          
 10  finish         3084 non-null   string        
 11  position       3084 non-null   float64       
 12  made_cut       3122 non-null   bool          
 13  rounds_played  3122 non-null   int64         
 14  strokes        3122 non-null   int64         
 15  sg_putt        2971 n

### Issue with `sg_xxxx`, `position`, and `finish` columns

As we can see, the `sg` fields have missing values for about 151 rows and there is no way to replace this data using the data in the DataFrame. For example, we could try to replace a certain player's `sg` null values with that player's average for that season, but this would be inaccurate information. This is the same case with `NaN` values in the position and finish columns.
**These rows will therefore have to be removed.**

In [8]:
pga_data = pga_data[(pga_data.sg_total.notnull()) & (pga_data.position.notnull()) & (pga_data.finish.notnull())]
print(pga_data.sample(10))
print(pga_data.info())
print(pga_data.describe())
print('Number of unique tournaments played:',pga_data['tournament'].nunique())
print('Number of unique players:',pga_data.player.nunique())

     player_id          player tournament_id                    tournament  \
390       3448  Dustin Johnson     401353227             AT&T Byron Nelson   
649       3599    Brian Stuard     401353229                   Mexico Open   
3533      3532  William McGirt     401353194  Sanderson Farms Championship   
3217      3550   Gary Woodland     401353196           The CJ Cup @ Summit   
260       3448  Dustin Johnson     401353226              PGA Championship   
118      10906       Aaron Rai     401353225      Charles Schwab Challenge   
3426       774       Bill Haas     401353194  Sanderson Farms Championship   
123       1225    Brian Harman     401353225      Charles Schwab Challenge   
1204      1225    Brian Harman     401353255          Valspar Championship   
2370     11333    Greyson Sigg     401353233          The American Express   

           date                       course           city   state  purse  \
390  2022-05-15              TPC Craig Ranch       McKinney    

We have a total of 20 columns, 8 `str` type, 10 `numeric` type where we can do our analysis, and 2 of `bool` type.

## YAYYYY!

#### We are now ready to perform analysis on our clean and tidy dataset!

In [9]:
pga_data.to_csv('pgatour_22_cleaned.csv')
%store pga_data

Stored 'pga_data' (DataFrame)
